書籍[pythonで作る対話システム]を基に言語モデルの学習を行うソースコード

In [ ]:
!python --version

In [ ]:
!pip check

In [ ]:
!pip install --upgrade pip
!pip install setuptools wheel
!pip3 install torch torchvision torchaudio
!pip install spacy panel==0.6.4 prophet holoviews==1.12.4 configargparse
# !pip install OpenNMT-py
!git clone https://github.com/OpenNMT/OpenNMT-py.git -b 1.0.0
!pip install -U OpenNMT-py


データの読み込みと保存のためGoogle Driveに接続します．

「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」と表示されるので「Google Driveに接続」をクリックし，使用するアカウントを選択します．

次のページで「許可」ボタンをクリックしてください．


In [ ]:
from google.colab import drive
drive.mount('./drive')

In [ ]:
!pip install torchtext==0.6.0

学習を実行します．

In [ ]:
!ls  "drive/My Drive/dsbook"

語彙辞書の作成

In [ ]:
!onmt_build_vocab --config "drive/My Drive/dsbook/config_chat_bot.yaml" --n_sample -1

In [ ]:
!head "drive/My Drive/dsbook/OpenNMT/vocab.txt" -n 20

RNN(LSTM)の学習1

In [ ]:
!onmt_train --config "drive/My Drive/dsbook/test.yaml"

学習結果の確認

In [ ]:
!onmt_translate -model "drive/My Drive/dsbook/OpenNMT/dlg_model_step_1500.pt"  --src "drive/My Drive/dsbook/OpenNMT/test.src" --output "drive/My Drive/dsbook/OpenNMT/pred_1500_by_lstm1.txt"

RNN(LSTM)の学習2

In [ ]:
!python OpenNMT-py/preprocess.py -train_src "drive/My Drive/dsbook/OpenNMT/train.src" -train_tgt "drive/My Drive/dsbook/OpenNMT/train.tgt" \
 -valid_src "drive/My Drive/dsbook/OpenNMT/dev.src" -valid_tgt "drive/My Drive/dsbook/OpenNMT/dev.tgt" -save_data dlg
!python OpenNMT-py/train.py -gpu_ranks 0 --save_checkpoint_steps 500 --train_steps 3000 -save_model "drive/My Drive/dsbook/OpenNMT/lstm_model2" -data dlg

学習結果の表示。

In [ ]:
!python OpenNMT-py/translate.py -model "drive/My Drive/dsbook/OpenNMT/lstm_model2_step_1500.pt" -src "drive/My Drive/dsbook/OpenNMT/test.src"  -output "drive/My Drive/dsbook/OpenNMT/pred_1500_by_lstm2.txt"  -replace_unk -verbose

transformerモデルの学習

In [ ]:
!onmt_train --config "drive/My Drive/dsbook/config_chat_bot_by_transformer.yaml"

In [ ]:
!onmt_translate -model "drive/My Drive/dsbook/OpenNMT/tsf_model_step_2000.pt"  --src "drive/My Drive/dsbook/OpenNMT/test.src" --output "drive/My Drive/dsbook/OpenNMT/pred_2000_by_tsf.txt"

rinna gpt2(apanese-gpt2-medium)を利用したファインチューニング
学習用データとしてmanga109のデータ使用

パッケージのインストール

In [ ]:
# パッケージのインストール
!git clone https://github.com/huggingface/transformers -b v4.23.1
!pip install transformers==4.23.1
!pip install evaluate==0.3.0
!pip install sentencepiece==0.1.97

In [ ]:
!pip list

rinna gpt2(apanese-gpt2-medium)のファインチューニング

In [ ]:
!python ./transformers/examples/pytorch/language-modeling/run_clm.py     --model_name_or_path=rinna/japanese-gpt2-small     --train_file="dialogue_pairs_re.txt"     --do_train     --do_eval     --num_train_epochs=10     --save_steps=10000     --block_size 512     --save_total_limit=3     --output_dir "./output"     --overwrite_output_dir     --use_fast_tokenizer=False

In [ ]:
!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-small \
    --train_file="drive/My Drive/dsbook/OpenNMT/siraisi_dialogue_re.txt" \
    --do_train \
    --do_eval \
    --num_train_epochs=10 \
    --save_steps=10000 \
    --block_size 512 \
    --save_total_limit=3 \
    --output_dir "drive/My Drive/output_by_siraisi/" \
    --overwrite_output_dir \
    --use_fast_tokenizer=False

学習用データを変えてファインチューニングを実験

In [ ]:
from transformers import T5Tokenizer, AutoModelForCausalLM

# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-small")
# 訳7万件のテキストデータでファインチューニングしたもの
model = AutoModelForCausalLM.from_pretrained("drive/My Drive/output")
# 訳3万件の対話テキストデータでファインチューニングしたもの
# model = AutoModelForCausalLM.from_pretrained("drive/My Drive/output_by_dialogue_pairs_re/")
# manga109の漫画ラブひなの役3000件のテキストデータでファインチューニングしたもの
# model = AutoModelForCausalLM.from_pretrained("drive/My Drive/output_by_LoveHina/")
# 上記のテキストデータのうち、発話者がnaruのものだけを抽出してファインチューニングしたもの
# model = AutoModelForCausalLM.from_pretrained("drive/My Drive/output_LoveHina_by_naru/")
# 無理やり2万件にしたもの
# model = AutoModelForCausalLM.from_pretrained("drive/My Drive/output_LoveHina_by_naru_inc_10000/")
# 無理やり1万件にしたもの
# model = AutoModelForCausalLM.from_pretrained("drive/My Drive/output_LoveHina_by_naru_inc_20000/")
# ラインのトーク履歴を使用
# model = AutoModelForCausalLM.from_pretrained("drive/My Drive/output_by_siraisi/")

In [ ]:
tokenizer.special_tokens_map

結果(ファインチューニング後)

In [ ]:
# 推論の実行
input = tokenizer.encode("こんちには", return_tensors="pt")
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=1)
# output = model.generate(input,do_sample=True,max_length=64, num_return_sequences=1,top_p=0.95, top_k=20, bad_words_ids=[[1], [5]], no_repeat_ngram_size=3)
print(tokenizer.batch_decode(output))

結果(ファインチューニングなし)

In [ ]:
from transformers import T5Tokenizer, AutoModelForCausalLM

# トークナイザーとモデルの準備
tokenizer_ = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model_ = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

In [ ]:
# 推論の実行
input = tokenizer_.encode("こんばんわ", return_tensors="pt")
output = model_.generate(input, do_sample=True, max_length=100, num_return_sequences=8)
print(tokenizer_.batch_decode(output))

bilingual-gpt-neox-4b-instruction-sftを利用したファインチューニング(グラボのram不足のため実行不可)

In [ ]:
!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path=rinna/bilingual-gpt-neox-4b-instruction-sft \
    --train_file="drive/My Drive/dsbook/OpenNMT/siraisi_dialogue_re.txt" \
    --do_train \
    --do_eval \
    --num_train_epochs=10 \
    --save_steps=10000 \
    --block_size 512 \
    --save_total_limit=3 \
    --output_dir "drive/My Drive/output_by_siraisi/" \
    --overwrite_output_dir \
    --use_fast_tokenizer=False

結果

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("rinna/bilingual-gpt-neox-4b-instruction-sft", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("rinna/bilingual-gpt-neox-4b-instruction-sft")

prompt = "こんにちは"

if torch.cuda.is_available():
    model = model.to("cuda")

token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=512,
        do_sample=True,
        temperature=1.0,
        top_p=0.85,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])
print(output)